In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import os.path as pth
import openmdao.api as om
import logging
import shutil
import fastoad.api as oad


DATA_FOLDER_PATH = "data"

WORK_FOLDER_PATH = "workdir"
CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, "oad_sizing.yml")
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, "CeRAS01_baseline.xml")

# For having log messages on screen
logging.basicConfig(level=logging.INFO, format="%(levelname)-8s: %(message)s")

# For using all screen width
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
shutil.copy(pth.join(DATA_FOLDER_PATH, "oad_sizing.yml"), CONFIGURATION_FILE)

In [ ]:
oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

In [ ]:
conf = oad.FASTOADProblemConfigurator("data/oad_sizing.yml") # reads the configuration file
#conf.write_needed_inputs("data/CeRAS01_baseline.xml") # you may skip this step if you already generated your input file
problem = conf.get_problem(read_inputs=True) # here you get an OpenMDAO problem (with only a few additions)
problem.setup()

In [ ]:
# Here you may add your recorder using the OpenMDAO way
recorder = om.SqliteRecorder("record.sql")
problem.model.nonlinear_solver.add_recorder(recorder)
problem.model.nonlinear_solver.recording_options['includes'] = ['*']


In [ ]:
# You run the problem normally
#problem1.setup()
problem.run_model()

In [ ]:
# To get FAST-OAD outputs, you have to use the method we have added
# to our slightly modified Problem class: 
problem.write_outputs()

In [ ]:
problem.record("final_state")

cr = om.CaseReader("record.sql")

solver_cases = cr.list_cases("root.nonlinear_solver", out_stream=None)

In [ ]:
for case_id in solver_cases:
    case = cr.get_case(case_id)
    print(case["data:weight:aircraft:MTOW"])

In [ ]:
#oad.generate_configuration_file(CONFIGURATION_FILE, overwrite=True)

# The above generated configuration uses the quick and simple Breguet module to compute performances.
# If you want to use the more accurate, much more CPU-costly time-step integration, uncomment and run
# the next line:
shutil.copy(pth.join(DATA_FOLDER_PATH, "oad_23.yml"), CONFIGURATION_FILE)

In [ ]:
oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

In [ ]:
oad.list_variables(CONFIGURATION_FILE)

In [ ]:
XDSM_FILE = pth.join(WORK_FOLDER_PATH, 'xdsm.html')
oad.write_xdsm(CONFIGURATION_FILE, XDSM_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=XDSM_FILE, width='100%', height='500px')

In [ ]:
eval_problem = oad.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

In [ ]:
oad.variable_viewer(eval_problem.output_file_path)

In [ ]:
fig = oad.wing_geometry_plot(eval_problem.output_file_path, name="FAST-OAD")
fig.show()

In [ ]:
oad.aircraft_geometry_plot(eval_problem.output_file_path)

In [ ]:
N2_FILE = pth.join(WORK_FOLDER_PATH, "n2.html")
oad.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame

IFrame(src=N2_FILE, width="100%", height="500px")

In [ ]:
XDSM_FILE = pth.join(WORK_FOLDER_PATH, 'xdsm.html')
oad.write_xdsm(CONFIGURATION_FILE, XDSM_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=XDSM_FILE, width='100%', height='500px')